In [ ]:
# N BEATS- 15 YEAR NO SENTIMENT 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, subtract, add
from tensorflow.keras.models import Model

# ----------------------------
# 1. Data Preparation (Univariate: 'close' price only)
# ----------------------------
# Load BTC data and sort by datetime
df_btc = pd.read_csv('/notebooks/BTCDATA.csv', parse_dates=['datetime'])
df_btc.sort_values('datetime', inplace=True)
df_btc = df_btc[['datetime', 'close', 'volume']]
print(df_btc.head())

# Split into training and test sets
train_df = df_btc[(df_btc['datetime'] >= '2011-01-01') & (df_btc['datetime'] < '2023-12-31')].copy()
test_df = df_btc[df_btc['datetime'] >= '2024-01-01'].copy()

# Use separate scaler for the 'close' price
close_scaler = MinMaxScaler()

# Fit and transform using training data; then transform test data
train_df['close_scaled'] = close_scaler.fit_transform(train_df[['close']])
test_df['close_scaled'] = close_scaler.transform(test_df[['close']])

# ----------------------------
# 2. Creating Sequences for N-BEATS (Univariate)
# ----------------------------
lookback = 30  # use the previous 30 days to predict the next day's close

def create_sequences_univariate(series, lookback):
    X, y = [], []
    for i in range(lookback, len(series)):
        X.append(series[i - lookback:i])
        y.append(series[i])
    return np.array(X), np.array(y)

# Create sequences for training and testing using only the 'close_scaled' series
X_train_univ, y_train_univ = create_sequences_univariate(train_df['close_scaled'].values, lookback)
X_test_univ, y_test_univ = create_sequences_univariate(test_df['close_scaled'].values, lookback)

# ----------------------------
# 3. Building a Simplified N-BEATS Model in Keras
# ----------------------------
# N-BEATS is designed for univariate forecasting.
# We set the backcast length equal to our lookback and forecast horizon to 1 (predict next-day price).

backcast_length = lookback
forecast_length = 1

def nbeats_block(backcast_length, forecast_length, n_neurons=128, n_layers=3):
    """
    Creates a single N-BEATS block.
    """
    input_block = Input(shape=(backcast_length,))
    x = input_block
    for _ in range(n_layers):
        x = Dense(n_neurons, activation='relu')(x)
    theta = Dense(backcast_length + forecast_length)(x)
    # Split the theta vector into backcast and forecast components
    backcast = Lambda(lambda x: x[:, :backcast_length])(theta)
    forecast = Lambda(lambda x: x[:, backcast_length:])(theta)
    return Model(inputs=input_block, outputs=[backcast, forecast])

# Number of blocks to stack
n_blocks = 3

# Input layer for N-BEATS: a vector of length 'backcast_length'
input_layer = Input(shape=(backcast_length,))
residual = input_layer  # initialize the residual to be the input
forecast_outputs = []

# Stack N-BEATS blocks
for i in range(n_blocks):
    block = nbeats_block(backcast_length, forecast_length, n_neurons=128, n_layers=3)
    backcast, forecast = block(residual)
    # Update residual by subtracting the block's backcast
    residual = subtract([residual, backcast])
    forecast_outputs.append(forecast)

# Sum all forecast outputs from the blocks to obtain the final forecast
final_forecast = add(forecast_outputs)
model_nbeats = Model(inputs=input_layer, outputs=final_forecast)
model_nbeats.compile(optimizer='adam', loss='mse')
model_nbeats.summary()

# ----------------------------
# 4. Training the N-BEATS Model
# ----------------------------
history_nbeats = model_nbeats.fit(
    X_train_univ, y_train_univ,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# ----------------------------
# 5. Prediction, Inverse Transformation, and Evaluation
# ----------------------------
# Generate predictions on the test set
predictions_nbeats_scaled = model_nbeats.predict(X_test_univ)
# Inverse transform to get predictions in original scale
predictions_nbeats = close_scaler.inverse_transform(predictions_nbeats_scaled)
y_test_actual_nbeats = close_scaler.inverse_transform(y_test_univ.reshape(-1, 1))

# Align dates for plotting (note that we lose the first 'lookback' days)
test_dates = test_df['datetime'].values[lookback:]

plt.figure(figsize=(12, 6))
plt.plot(test_dates, y_test_actual_nbeats, label='Actual Close Price', color='blue')
plt.plot(test_dates, predictions_nbeats, label='Predicted Close Price (N-BEATS)', color='red', linestyle='--')
plt.xlabel('Date')
plt.ylabel('BTC Close Price')
plt.title('BTC Close Price Prediction using N-BEATS: Actual vs Predicted')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate performance metrics
mse_nbeats = mean_squared_error(y_test_actual_nbeats, predictions_nbeats)
mae_nbeats = mean_absolute_error(y_test_actual_nbeats, predictions_nbeats)
rmse_nbeats = np.sqrt(mse_nbeats)
mape_nbeats = mean_absolute_percentage_error(y_test_actual_nbeats, predictions_nbeats) * 100
r2_nbeats = r2_score(y_test_actual_nbeats, predictions_nbeats)

print("Performance Metrics for the N-BEATS Model:")
print(f"MSE: {mse_nbeats:.2f}")
print(f"MAE: {mae_nbeats:.2f}")
print(f"RMSE: {rmse_nbeats:.2f}")
print(f"MAPE: {mape_nbeats:.2f}%")
print(f"R2 Score: {r2_nbeats:.2f}")


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, subtract, add
from tensorflow.keras.models import Model

# ----------------------------
# 1. Data Preparation (Univariate: 'close' price only)
# ----------------------------
# Load BTC data and sort by datetime
df_btc = pd.read_csv('/notebooks/BITCOIN.csv', parse_dates=['datetime'])
df_btc.sort_values('datetime', inplace=True)
# df_btc = df_btc[df_btc['datetime'] <= '2019-05-01'].copy()
df_btc = df_btc[['datetime', 'close', 'volume']]
print(df_btc.head())

# Split into training and test sets
train_df = df_btc[(df_btc['datetime'] >= '2011-01-01') & (df_btc['datetime'] < '2024-02-01')].copy()
test_df = df_btc[df_btc['datetime'] >= '2024-02-01'].copy()

# Use separate scaler for the 'close' price
close_scaler = MinMaxScaler()

# Fit and transform using training data; then transform test data
train_df['close_scaled'] = close_scaler.fit_transform(train_df[['close']])
test_df['close_scaled'] = close_scaler.transform(test_df[['close']])

# ----------------------------
# 2. Creating Sequences for N-BEATS (Univariate)
# ----------------------------
lookback = 30  # use the previous 30 days to predict the next day's close

def create_sequences_univariate(series, lookback):
    X, y = [], []
    for i in range(lookback, len(series)):
        X.append(series[i - lookback:i])
        y.append(series[i])
    return np.array(X), np.array(y)

# Create sequences for training and testing using only the 'close_scaled' series
X_train_univ, y_train_univ = create_sequences_univariate(train_df['close_scaled'].values, lookback)
X_test_univ, y_test_univ = create_sequences_univariate(test_df['close_scaled'].values, lookback)

# ----------------------------
# 3. Building a Simplified N-BEATS Model in Keras
# ----------------------------
# N-BEATS is designed for univariate forecasting.
# We set the backcast length equal to our lookback and forecast horizon to 1 (predict next-day price).

backcast_length = lookback
forecast_length = 1

def nbeats_block(backcast_length, forecast_length, n_neurons=128, n_layers=3):
    """
    Creates a single N-BEATS block.
    """
    input_block = Input(shape=(backcast_length,))
    x = input_block
    for _ in range(n_layers):
        x = Dense(n_neurons, activation='relu')(x)
    theta = Dense(backcast_length + forecast_length)(x)
    # Split the theta vector into backcast and forecast components
    backcast = Lambda(lambda x: x[:, :backcast_length])(theta)
    forecast = Lambda(lambda x: x[:, backcast_length:])(theta)
    return Model(inputs=input_block, outputs=[backcast, forecast])

# Number of blocks to stack
n_blocks = 3

# Input layer for N-BEATS: a vector of length 'backcast_length'
input_layer = Input(shape=(backcast_length,))
residual = input_layer  # initialize the residual to be the input
forecast_outputs = []

# Stack N-BEATS blocks
for i in range(n_blocks):
    block = nbeats_block(backcast_length, forecast_length, n_neurons=128, n_layers=3)
    backcast, forecast = block(residual)
    # Update residual by subtracting the block's backcast
    residual = subtract([residual, backcast])
    forecast_outputs.append(forecast)

# Sum all forecast outputs from the blocks to obtain the final forecast
final_forecast = add(forecast_outputs)
model_nbeats = Model(inputs=input_layer, outputs=final_forecast)
model_nbeats.compile(optimizer='adam', loss='mse')
model_nbeats.summary()

# ----------------------------
# 4. Training the N-BEATS Model
# ----------------------------
history_nbeats = model_nbeats.fit(
    X_train_univ, y_train_univ,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# ----------------------------
# 5. Prediction, Inverse Transformation, and Evaluation
# ----------------------------
# Generate predictions on the test set
predictions_nbeats_scaled = model_nbeats.predict(X_test_univ)
# Inverse transform to get predictions in original scale
predictions_nbeats = close_scaler.inverse_transform(predictions_nbeats_scaled)
y_test_actual_nbeats = close_scaler.inverse_transform(y_test_univ.reshape(-1, 1))

# Align dates for plotting (note that we lose the first 'lookback' days)
test_dates = test_df['datetime'].values[lookback:]

plt.figure(figsize=(12, 6))
plt.plot(test_dates, y_test_actual_nbeats, label='Actual Close Price', color='blue')
plt.plot(test_dates, predictions_nbeats, label='Predicted Close Price (N-BEATS)', color='red', linestyle='--')
plt.xlabel('Date')
plt.ylabel('BTC Close Price')
plt.title('BTC Close Price Prediction using N-BEATS: Actual vs Predicted')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Calculate performance metrics
mse_nbeats = mean_squared_error(y_test_actual_nbeats, predictions_nbeats)
mae_nbeats = mean_absolute_error(y_test_actual_nbeats, predictions_nbeats)
rmse_nbeats = np.sqrt(mse_nbeats)
mape_nbeats = mean_absolute_percentage_error(y_test_actual_nbeats, predictions_nbeats) * 100
r2_nbeats = r2_score(y_test_actual_nbeats, predictions_nbeats)

print("Performance Metrics for the N-BEATS Model:")
print(f"MSE: {mse_nbeats:.2f}")
print(f"MAE: {mae_nbeats:.2f}")
print(f"RMSE: {rmse_nbeats:.2f}")
print(f"MAPE: {mape_nbeats:.2f}%")
print(f"R2 Score: {r2_nbeats:.2f}")
